## Homework 5 - Mini Challenge Warm-Up
#### Authors: Ajay Shaan Shanmugam, Siddharth Chandrasekaran

# Problem statement

For this homework, we have chosen the VAST 2008 mini challenge 1 which involves identifying the social network of specific persons of interest using visual analytics. The call record dataset provided includes information on the caller, receiver, the duration of call, the corresponding timestamp and the cell tower through which the call was made. Please note that the caller and receiver records do not have useful identifiers. i.e They use numbers instead of actual names. Using provided bits of information on these suspects, we undertake the task of identifying them using a network graph, clustering the dataset using unsupervised learning techniques and gaining insights from their call patterns over the 10 day period. We aren't using the cell tower information for this mini challenge since the geo location data is relevant only to solve the grand challenge. 

Although quality CDRs (Call Data Records) similar to our dataset were hard to come by, we finalized on using a subset of the Enron email dataset, with only features that were of interest and in a way mirrored our mini challenge dataset. The Enron email dataset consists of 500000+ emails sent and received between 150 employees of the once-mighty Enron Corporation and outside sources. Although the original dataset contains text information on which natural language processing techniques are applied typically, in order to draw parallels between out mini challenge dataset and this, we devised an approach to determine importance of email communication between two individuals so as to align with the weights that we estimate for our mini challenge CDR (Call duration and frequency). 

Enron dataset is widely researched upon from NLP point of view (since there is a lot of textual data) and to analyse the social relationships between the folks involved in the dataset based on frequency and length of the communications. As mentioned earlier we are interested in the latter since it aligns more with the mini challenge. Majority of the approaches visualized the dataset using graphs and used clustering algorithms on the nodes (employees) to find people who communicate with each other (Communication is represented as the edge). We decided to use clustering algorithms such as triangles and square clustering for this dataset.


# About the visualization

Since Bokeh doesnt have default support to make graphs (it has an option to render networkx graphs but it is not possible to add interactive visualizations and widgets to the dataset). We had to improvise and create a graph using the basic glyphs of bokeh such as a circle for the node and a line for edges (with thickness based on number of emails, importance of the email and the length of the message body).

Since we needed the graph to be well aligned and evenly spaced out we decided to optimize the modularity of the graph using the louvain technique. This ensures that the graph gets divided into "communities" based on the density of the communications and that these communities are evenly spaced out. We found a python module called python-louvain which can do this for us and used it to get the coordinates of the edges and the nodes which can be plotted in bokeh.

We added a widget to give the viewer the option to color code nodes based on clusters or to display all the nodes in a single color (to look at the data devoid of the bias clustering algorithms give us). This graph helps us identify patterns in the network, inspect clusters, individuals (using hover).

This graph gave us a lot of useful insights on the relationships of the enron employees and their outside sources based on email exchanges. We were able to identify the most common mailers, folks who work closely with each other etc.

We used three clustering algorithms namely
1. Triangle based clustering
2. Clustering based on geometric average of the subgraph edge weights
3. Square clustering

and displayed all of them one after the other (bokeh doesnt have the support to visualize them side by side on different tabs). We didnt want to add a drop down box since we wanted to compare and contrast the algorithms at the same time.

For this mini challenge, we intend to use the network graph in addition to other plots for exploratory visualization, and the aforementioned clustering algorithms to cluster nodes and make inferences.

# Run instructions:

As mentioned above due to bokeh's limitations we had to install an extra package called python louvian to construct the social network graph

So you will need this package installed to run this notebook.

pip install python-louvain

### Note: 
The importance of a communication determines its weight and this is proportional to the thickness of the corresponding edge in the network graph

In [3]:
__authors__ = "Ajay Shaan Shanmugam;Siddharth Chandrasekaran"
__license__ = "GPL"
__version__ = "1.0.1"
__emails__ = "ashanmugam@umass.edu;schandraseka@umass.edu"

import pandas as pd

from random import randint

import networkx as nx

from collections import namedtuple

from math import sqrt

import bokeh

from bokeh.models import HoverTool

from bokeh.plotting import show, figure

from bokeh.colors import RGB

import random

import community.community_louvain as community

data = pd.read_csv('emails.csv')
data1 = data.iloc[0:1000]
app = data.iloc[517101:517400]
data1=data1.append(app)
data1.append(data.iloc[317101:317401])
#Taking only a subset due to rendering constraints
data = data1

weights = []

for index,row in data.iterrows():
    message = ''
    for i in row['message'].split('\n')[16:]:
        message = message+i
    weights.append(len(message))
    
senders = []
receivers = []

for index,row in data.iterrows():
    sender = row['message'].split('\n')[2][6:]
    receiver = row['message'].split('\n')[3][4:]
    senders.append(sender)
    if receiver.find(','):
        receiver = receiver.split(',')
        receiver = receiver[0]
    receivers.append(receiver)
    
edgelist = pd.DataFrame({'senders':senders,'receivers':receivers,'weights':weights})
#Normalizing for displaying
edgelist['weights'] = edgelist['weights']/100

all_nodes = list(set(senders))

for rec in list(set(receivers)):
    if rec in all_nodes:
        continue
    else:
        all_nodes.append(rec)
        


def generate_coordinates(m, n):
    seen = set()

    x, y = randint(m, n), randint(m, n)

    while True:
        seen.add((x, y))
        yield [x, y]
        x, y = randint(m, n), randint(m, n)
        while (x, y) in seen:
            x, y = randint(m, n), randint(m, n)

g = generate_coordinates(1,2598)
xs = []
ys = []

for i in range(len(all_nodes)):
    xy = next(g)
    x = xy[0]
    y = xy[1]
    xs.append(x)
    ys.append(y)
nodelist = pd.DataFrame({'nodes':all_nodes,'x':xs,'y':ys})


def create_bokeh_graph(graph, clus_algo):

    

    def gen_edge_coordinates(graph, layout):

        xs = []

        ys = []

        val = namedtuple("edges", "xs ys")

        for edge in graph.edges():

            from_node = layout[edge[0]]

            to_node = layout[edge[1]]

            xs.append([from_node[0],to_node[0]])

            ys.append([from_node[1], to_node[1]])

        return val(xs=xs, ys=ys)



    def gen_node_coordinates(layout):

        names, coords = zip(*layout.items())

        xs, ys = zip(*coords)

        val = namedtuple("nodes", "names xs ys")

        return val(names=names, xs=xs, ys=ys)



    plot_layout = nx.spring_layout(graph)

    edges = graph.edges()
    
    weights = [graph[u][v]['weight'] for u,v in edges]

    nx.draw(graph, plot_layout, edges=edges, width=weights)

    _nodes = gen_node_coordinates(plot_layout)

    _edges = gen_edge_coordinates(graph, plot_layout)


    hover = HoverTool(tooltips=[('name', '@name'), 

                                ('node_id', '$index'),

                                ('degree', '@degree'),

                                ('cluster_no', '@community_nr')], names=["show_hover"])



    plot = figure(width=800, height=600, 

                 tools=[hover, 'box_zoom', 'resize', 'reset', 'wheel_zoom', 'pan', 'lasso_select'],

                logo = None)
    
    plot.title.text ="Algorithm : " + clus_algo

    plot.toolbar.logo = None

    plot.axis.visible = False                            

    plot.xgrid.grid_line_color = None

    plot.ygrid.grid_line_color = None

    

    #Draw Edges

    source_edges = bokeh.models.ColumnDataSource(dict(xs=_edges.xs, ys=_edges.ys))

    plot.multi_line('xs', 'ys', line_color='navy', source=source_edges, alpha=0.17, line_width=weights)

    

    #best partition based on the algorithm selected

    degrees = list(nx.degree(graph).values())
    
    if clus_algo == "triangle":
        
        clustering = list(nx.triangles(graph).values())
    
    elif clus_algo == "square":
        
        clustering = list(nx.square_clustering(graph).values())
    
    else:
        
        clustering = list(nx.clustering(graph).values())
        
    best_partition = community.best_partition(graph)

    nodes, communities = zip(*best_partition.items())

    betw = list(nx.betweenness_centrality(graph).values())

    

    #Color mapper

    colormapper = {x : RGB(random.randrange(0,256),random.randrange(0,256),random.randrange(0,256)) 

                     for x in list(set(communities))}

    cluster_color_list, community_nr = zip(*[(colormapper[best_partition[node]], best_partition[node]) for node in nodes])



    

    graph_nodes = graph.number_of_nodes()

    

    colors = ['firebrick' for node in range(graph_nodes)]

    

    #Drawing circles for nodes

    source_nodes = bokeh.models.ColumnDataSource(dict(xs=_nodes.xs, ys=_nodes.ys, name=_nodes.names, 

                                                      single_color = colors,

                                                      color_by_cluster = cluster_color_list, 

                                                      degree=degrees, 

                                                      clustering=clustering, community_nr=community_nr,

                                                      betweenness = betw))

    

    r_circles = plot.circle('xs', 'ys', fill_color='single_color', line_color='single_color', 

                           source = source_nodes, alpha=0.7, size=9, name="show_hover")

    
 

    colorcallback = bokeh.models.callbacks.CustomJS(args=dict(source=source_nodes, circles=r_circles), code="""

        var value = cb_obj.get('value');

        circles.glyph.line_color.field = value;

        circles.glyph.fill_color.field = value;

        source.trigger('change')

    """)  

    

    button = bokeh.models.widgets.Select(title="Color", value="single_color", 

                                         options=["single_color", "color_by_cluster"], 

                                         callback=colorcallback)

    
    return [plot,button]


In [4]:
g = nx.Graph()

for i, elrow in edgelist.iterrows():
    g.add_edge(elrow['senders'], elrow['receivers'], weight=elrow['weights'])
    
for i, nlrow in nodelist.iterrows():
    g.node[nlrow['nodes']] = nlrow[1:].to_dict()

layout_plot = []
layout_plot.append(create_bokeh_graph(g, "triangle"))
layout_plot.append(create_bokeh_graph(g, "square"))
layout_plot.append(create_bokeh_graph(g, "clustering"))
    #Create grid and save

layout_plot = bokeh.layouts.gridplot(layout_plot)
show(layout_plot)

D:\Anaconda\lib\site-packages\networkx\drawing\nx_pylab.py:126: MatplotlibDeprecationWarning: pyplot.hold is deprecated.
    Future behavior will be consistent with the long-time default:
    plot commands add elements without first clearing the
    Axes and/or Figure.
  b = plt.ishold()
D:\Anaconda\lib\site-packages\networkx\drawing\nx_pylab.py:138: MatplotlibDeprecationWarning: pyplot.hold is deprecated.
    Future behavior will be consistent with the long-time default:
    plot commands add elements without first clearing the
    Axes and/or Figure.
  plt.hold(b)
D:\Anaconda\lib\site-packages\matplotlib\__init__.py:917: UserWarning: axes.hold is deprecated. Please remove it from your matplotlibrc and/or style files.
  warnings.warn(self.msg_depr_set % key)
D:\Anaconda\lib\site-packages\matplotlib\rcsetup.py:152: UserWarning: axes.hold is deprecated, will be removed in 3.0
  warnings.warn("axes.hold is deprecated, will be removed in 3.0")
D:\Anaconda\lib\site-packages\bokeh\util\depr